In [39]:
import glob, os
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import product
import matplotlib.pyplot as plt
import utils

In [40]:
SERVER = 'gl2'

In [41]:
DATAPATH = f"../experiments/{SERVER}_data.csv"

In [42]:
data = pd.read_csv(DATAPATH)

In [43]:
subjects = list(set(data['subject'].values))
targets = list(set(data['target'].values))

GROUPS = list(
    product(subjects, targets)
)

In [44]:
data['subject'] = pd.Categorical(
    data['subject'],
    ['cpython', 'pypy', 'numba', 'pyston-lite', 'py3.13-jit', 'nuitka', 'cython', 'codon', 'mypyc']
)

In [45]:
# Convert to KJ and add Total Energy
data['PP0_ENERGY (J)'] = data['PP0_ENERGY (J)'] / 1000
data['DRAM_ENERGY (J)'] = data['DRAM_ENERGY (J)'] / 1000
data['TOTAL_ENERGY'] = data['PP0_ENERGY (J)'] + data['DRAM_ENERGY (J)']
data.rename(columns={'PP0_ENERGY (J)': 'PP0_ENERGY (KJ)', 'DRAM_ENERGY (J)': 'DRAM_ENERGY (KJ)'}, inplace=True)

In [46]:
data['execution_time'] = data['execution_time'] / 60
data['USED_MEMORY'] = data['USED_MEMORY'] / 1000000000

In [47]:
METRICS_INTEREST = ['TOTAL_ENERGY', 'execution_time', 'USED_MEMORY', 'LLC-load-misses_percent']

In [48]:
def reform(dictionary):
    return {(outerKey, innerKey): values for outerKey, innerDict in dictionary.items() for innerKey, values in innerDict.items()}

In [49]:
descriptive = []
for subject in subjects:
    df = data[data['subject'] == subject]
    descriptive.append(
        reform(df[METRICS_INTEREST].describe().to_dict()) | {'subject' : subject}
    )

columns = pd.MultiIndex.from_product([METRICS_INTEREST, ['mean', 'std', 'min', '50%', 'max']])
table = pd.DataFrame(descriptive, columns = columns)
table[('subject', '')] = subjects

table['subject'] = pd.Categorical(
    table['subject'],
    ['cpython', 'pypy', 'numba', 'pyston-lite', 'py3.13-jit', 'nuitka', 'cython', 'codon', 'mypyc']
)

# move last column to first position
last_column = table.iloc[:, -1]
table = pd.concat([last_column, table.iloc[:, :-1]], axis=1)

# sort value according to the one fixed by us
table = table.sort_values(by='subject')

table.head()

subject TOTAL_ENERGY                                           \
                       mean       std       min       50%        max   
2      cpython     8.203128  6.524057  2.598171  5.424499  22.540498   
6         pypy     0.747930  0.427124  0.167895  0.810002   1.453917   
4        numba     0.640801  0.358773  0.108728  0.531816   1.268076   
0  pyston-lite     8.228030  6.547062  2.605602  5.459775  22.823402   
7   py3.13-jit     6.447696  3.959217  2.844232  5.079902  19.632044   

  execution_time                                  ... USED_MEMORY            \
            mean        std       min        50%  ...        mean       std   
2      25.533828  20.623406  8.123457  17.144353  ...    3.892732  0.363480   
6       2.358685   1.388681  0.585438   2.245310  ...    2.590259  1.303687   
4       2.145817   1.277018  0.382412   1.735417  ...    2.764716  1.361631   
0      25.644125  20.763091  8.142628  17.205505  ...    3.825779  0.525823   
7      20.232385  12.696134  8.880125  15.636411  ...    3.825098  0.489964   

                                LLC-load-misses_percent                       \
        min       50%       max                    mean        std       min   
2  1.909597  3.786582  4.536351               21.870340  24.433711  0.602926   
6  1.153736  2.691224  4.250462               13.246550  14.140134  0.154551   
4  1.216085  2.964177  4.672427               18.224406  14.528662  3.519317   
0  1.211929  3.780345  4.549308               21.414565  24.254621  0.590686   
7  1.200062  3.782379  4.255997               17.686159  25.005815  0.340023   

                         
         50%        max  
2  11.369548  74.536082  
6   5.530717  36.927923  
4  15.298604  53.296344  
0  11.624700  74.359021  
7   4.770176  73.814606  

[5 rows x 21 columns]

In [50]:
latex_table = table.round(3).to_latex(index=False, float_format="%.2f")
print(latex_table)

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrr}
\toprule
subject & \multicolumn{5}{r}{TOTAL_ENERGY} & \multicolumn{5}{r}{execution_time} & \multicolumn{5}{r}{USED_MEMORY} & \multicolumn{5}{r}{LLC-load-misses_percent} \\
 & mean & std & min & 50% & max & mean & std & min & 50% & max & mean & std & min & 50% & max & mean & std & min & 50% & max \\
\midrule
cpython & 8.20 & 6.52 & 2.60 & 5.42 & 22.54 & 25.53 & 20.62 & 8.12 & 17.14 & 72.17 & 3.89 & 0.36 & 1.91 & 3.79 & 4.54 & 21.87 & 24.43 & 0.60 & 11.37 & 74.54 \\
pypy & 0.75 & 0.43 & 0.17 & 0.81 & 1.45 & 2.36 & 1.39 & 0.58 & 2.25 & 4.77 & 2.59 & 1.30 & 1.15 & 2.69 & 4.25 & 13.25 & 14.14 & 0.15 & 5.53 & 36.93 \\
numba & 0.64 & 0.36 & 0.11 & 0.53 & 1.27 & 2.15 & 1.28 & 0.38 & 1.74 & 4.52 & 2.77 & 1.36 & 1.22 & 2.96 & 4.67 & 18.22 & 14.53 & 3.52 & 15.30 & 53.30 \\
pyston-lite & 8.23 & 6.55 & 2.61 & 5.46 & 22.82 & 25.64 & 20.76 & 8.14 & 17.21 & 73.13 & 3.83 & 0.53 & 1.21 & 3.78 & 4.55 & 21.41 & 24.25 & 0.59 & 11.62 & 74.36 \\
py3.13-jit & 6.45 & 3.96